# Importsand preparations

In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, DatasetDict

/itf-fi-ml/home/krimhau/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.1.0+cu121
CUDA version: 12.1
cuDNN version: 8902
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.35.0
Datasets version: 2.14.6


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Wed Mar  6 12:52:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:01:00.0 Off |                  N/A |
| 40%   66C    P2             248W / 250W |    715MiB / 11264MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 99% |  6% |
|  1 |  0% |  6% |
|  2 |  0% |  6% |
|  3 |  0% |  0% |
|  4 |  0% |  6% |
|  5 |  0% |  0% |
|  6 |  0% |  0% |
|  7 |  0% |  6% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 99% |  6% |
|  1 |  0% |  6% |
|  2 |  0% |  6% |
|  3 |  0% |  0% |
|  4 |  0% |  6% |
|  5 |  0% |  0% |
|  6 |  1% |  1% |
|  7 |  0% |  6% |


In [8]:
# Smaller and faster than bert.
model_ckpt = "distilbert-base-uncased"

epochs = 5 #Number of full cycles through the training set.
num_labels = 2 #Number of labels, high, med, low priority.
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evaluation examples in on iteration.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

Load dataset from huggingface

In [9]:
dataset = load_dataset("kristmh/hp_nhp_only_hp_in_label")
dataset

Generating validate split: 100%|████████████████████████████████████████████████████| 16534/16534 [00:00<00:00, 130405.05 examples/s]


DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 16535
    })
    train: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 132276
    })
    validate: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 16534
    })
})

In [10]:
rust = load_dataset("kristmh/rust_testset_with_med_low_1")
rust

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 1572
    })
})

In [11]:
flutter = load_dataset("kristmh/flutter_testset_with_med_low_1")
flutter

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 2370
    })
})

In [12]:
mypy = load_dataset("kristmh/mypy_testset_with_med_low_1")
mypy

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 734
    })
})

In [13]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

## Tokenization

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

    Tokenizing the whole dataset

In [17]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

In [18]:
rust_tokenized = rust.map(tokenize, batched=True)
rust_testset = rust_tokenized["test"]
rust_testset

Dataset({
    features: ['text_clean', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1572
})

In [19]:
flutter_tokenized = flutter.map(tokenize, batched=True)
flutter_testset = flutter_tokenized["test"]
flutter_testset

Dataset({
    features: ['text_clean', 'label', 'input_ids', 'attention_mask'],
    num_rows: 2370
})

In [20]:
mypy_tokenized = mypy.map(tokenize, batched=True)
mypy_testset = mypy_tokenized["test"]
mypy_testset

Dataset({
    features: ['text_clean', 'label', 'input_ids', 'attention_mask'],
    num_rows: 734
})

In [21]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map: 100%|████████████████████████████████████████████████████████████████████████████| 16534/16534 [00:07<00:00, 2087.46 examples/s]


DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 16535
    })
    train: Dataset({
        features: ['text_clean', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 132276
    })
    validate: Dataset({
        features: ['text_clean', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 16534
    })
})

In [22]:
# List the columns of the dataset.
# Should be: ["attention_mask", "input_ids", "labels", "token_type_ids"]
# Remove unnecessary columns that the model does not expect.
tokenized_dataset = tokenized_dataset.remove_columns(["text_clean", "__index_level_0__"])

train_dataset = tokenized_dataset["train"]
print(train_dataset)
validation_dataset = tokenized_dataset["validate"]
print(validation_dataset)
test_dataset = tokenized_dataset["test"]
test_dataset

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 132276
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 16534
})


Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 16535
})

## Training a classifier

In [23]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [25]:
trainer.train() 

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.620300,0.599847,0.669953,[0.63338932 0.69988451],[0.70274299 0.64756768],[0.57649505 0.76139763]
2,0.542200,0.611767,0.679751,[0.64143022 0.71067155],[0.71866464 0.65396219],[0.57918552 0.77815005]
3,0.501000,0.630331,0.688158,[0.66335858 0.70955385],[0.71158426 0.67035657],[0.62125474 0.75361972]
4,0.425500,0.669917,0.685859,[0.6907597 0.68080138],[0.67304792 0.69981049],[0.70942889 0.66279765]
5,0.313700,0.793428,0.681263,[0.67541266 0.68690589],[0.68035736 0.68212389],[0.67053932 0.69175541]


TrainOutput(global_step=41340, training_loss=0.4862972946056261, metrics={'train_runtime': 12868.4607, 'train_samples_per_second': 51.395, 'train_steps_per_second': 3.213, 'total_flos': 8.761128812310528e+16, 'train_loss': 0.4862972946056261, 'epoch': 5.0})

* Training loss: Difference between the predictons made by the model on the training dataset vs on the actual data.
* Validation loss: how well the model functions on unseen data.
* Accuracy: How much the model gets correct. number of correct Prediction / total number of predictions.
* F1: consider both precision and recall. 
* Precision: Accuracy of positive predictions. Percison TP = TP + FP. How often the model is correct.
* Recall: True positive rate. how many items the model gets correct from the total amount.

### Training loss decreases, validation loss increases = Overfitting

In [26]:
# Evaluate validation set
eval_result = trainer.evaluate(eval_dataset=validation_dataset)

In [27]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.6812628523043426

eval_f1 = [0.67541266 0.68690589]

eval_loss = 0.7934283018112183

eval_precision = [0.68035736 0.68212389]

eval_recall = [0.67053932 0.69175541]

eval_runtime = 103.8557

eval_samples_per_second = 159.202

eval_steps_per_second = 4.978



In [28]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [29]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.6802540066525552

eval_f1 = [0.67514593 0.68520393]

eval_loss = 0.79090416431427

eval_precision = [0.68657836 0.67432322]

eval_recall = [0.664088   0.69644154]

eval_runtime = 103.9124

eval_samples_per_second = 159.124

eval_steps_per_second = 4.975



In [30]:
rust_test_results = trainer.evaluate(eval_dataset=rust_testset)
for key, value in sorted(rust_test_results.items()):
    print(f"{key} = {value}\n")    

epoch = 5.0

eval_accuracy = 0.5865139949109415

eval_f1 = [0.57180501 0.600246  ]

eval_loss = 1.129926323890686

eval_precision = [0.59289617 0.58095238]

eval_recall = [0.55216285 0.62086514]

eval_runtime = 9.9689

eval_samples_per_second = 157.69

eval_steps_per_second = 5.016



In [31]:
flutter_test_results = trainer.evaluate(eval_dataset=flutter_testset)
for key, value in sorted(flutter_test_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.5236286919831223

eval_f1 = [0.60620858 0.39722371]

eval_loss = 1.4791343212127686

eval_precision = [0.51664685 0.54069767]

eval_recall = [0.73333333 0.31392405]

eval_runtime = 14.9679

eval_samples_per_second = 158.339

eval_steps_per_second = 5.011



In [32]:
mypy_test_results = trainer.evaluate(eval_dataset=mypy_testset)
for key, value in sorted(mypy_test_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.6198910081743869

eval_f1 = [0.69034406 0.50793651]

eval_loss = 1.0085598230361938

eval_precision = [0.582397 0.72    ]

eval_recall = [0.84741144 0.39237057]

eval_runtime = 4.6501

eval_samples_per_second = 157.845

eval_steps_per_second = 4.946



In [33]:
trainer.save_model(model_dir + "_local") 

In [34]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local")

In [35]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [36]:
classifier("this does not need to be done fast")

[{'label': 'LABEL_1', 'score': 0.7422532439231873}]

In [37]:
classifier("this is super important")

[{'label': 'LABEL_0', 'score': 0.9439973831176758}]

In [38]:
classifier("this bug has super high impact on the project")

[{'label': 'LABEL_0', 'score': 0.7985531687736511}]

## Important to delete large objects to free memory 
del train_dataset

In [39]:
del validation_dataset 

In [40]:
del model

In [41]:
# Free cache
torch.cuda.empty_cache()

In [42]:
!nvidia-smi

Wed Mar  6 16:32:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:01:00.0 Off |                  N/A |
| 39%   65C    P2             250W / 250W |    703MiB / 11264MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--